<a href="https://colab.research.google.com/github/KDvs123/cutom-ner-with-spacy3/blob/main/Name_Entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies


In [1]:
! pip install -U spacy -q


Data


In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.5                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [3]:
#convert the json file into the docbin object

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json
f=open('annotations.json')
TRAIN_DATA = json.load(f)

In [5]:
TRAIN_DATA

{'classes': ['ASSIGNEE', 'DUE-DATE', 'CREATED-DATE', 'TYPE', 'LABEL'],
 'annotations': [['Show tasks assigned to Alice',
   {'entities': [[23, 28, 'ASSIGNEE']]}],
  ['Find all tasks of type input', {'entities': [[23, 28, 'TYPE']]}],
  ['Get tasks due on 2023-10-11', {'entities': [[17, 27, 'DUE-DATE']]}],
  ['Retrieve tasks created on 2024-01-27',
   {'entities': [[26, 36, 'CREATED-DATE']]}],
  ['Filter tasks with the label Marketing', {'entities': [[28, 37, 'LABEL']]}],
  ['Show tasks assigned to Eva and of type approval',
   {'entities': [[23, 26, 'ASSIGNEE'], [39, 47, 'TYPE']]}],
  ['Find tasks with the label Projects due before 2024-01-01',
   {'entities': [[26, 34, 'LABEL'], [46, 56, 'DUE-DATE']]}],
  ['Get tasks created after 2023-01-01 for Bob',
   {'entities': [[24, 34, 'CREATED-DATE'], [39, 42, 'ASSIGNEE']]}],
  ['Retrieve tasks of type input assigned to Charlie',
   {'entities': [[23, 28, 'TYPE'], [41, 48, 'ASSIGNEE']]}],
  ['Show all tasks with due dates in 2024',
   {'entiti

In [7]:
for text , annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start , end , label in annot['entities']:
        span = doc.char_span(start,end,label=label,alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents =ents
    db.add(doc)

db.to_disk('./annotations.spacy') #save the docbin object

100%|██████████| 16/16 [00:00<00:00, 2634.92it/s]


In [8]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./annotations.spacy --paths.dev ./annotations.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     60.27    0.00    0.00    0.00    0.00
 66     200         50.03   1691.82  100.00  100.00  100.00    1.00
156     400          0.00      0.00  100.00  100.00  100.00    1.00
256     600          0.00      0.00  100.00  100.00  100.00    1.00
356     800          0.00      0.00  100.00  100.00  100.00    1.00
532    1000          0.00      0.00  100.00  100.00  100.00    1.00
732    1200          0.00      0.00  100.00  100.00  100.00    1.00
932    1400          0.00      0.00  100.00  100.00  100.00    1.00
1132    1600          0.00      0.00

In [10]:
nlp_ner =spacy.load ("/content/model-best")

In [20]:
doc = nlp_ner ('Show the tasks that was assigned to Vihanga that was created in 2024 under Management label')

In [21]:
spacy.displacy.render(doc,style="ent",jupyter=True) #display in  jupyter